In [1]:
import configparser

# Cria o parser e lê o arquivo
config = configparser.ConfigParser()
config.read('replicas.conf')

['replicas.conf']

In [4]:
from utils.transaction import Transaction

t = Transaction(cid=4050)
print(t.replica)

scan = t.scan(think_time=0)
print(scan)

t.write("teste", 1)
t.commit()

scan = t.scan(think_time=0)
print(scan)
print(f"TX1: {t.result}")


('localhost', 4001)
[Cliente 4050] Efetuando SCAN no datastore
[Unicast.request] Enviando para localhost:4001 a mensagem {'type': 'SCAN', 'cid': 4050}
{'gui': (123, 0), 'x': (100, 1), 'y': (30, 0), 'z': (50, 0)}
[Cliente 4050] Efetuando escrita na chave 'teste' com valor '1'
[Cliente 4050] Commitando transação 882f6960-3ad8-4a03-a4f4-65a20bffb47d
[Unicast.request] Enviando para localhost:4000 a mensagem {'type': 'COMMIT_REQ', 'cid': 4050, 'tid': '882f6960-3ad8-4a03-a4f4-65a20bffb47d', 'rs': [], 'ws': [('teste', 1)]}
[Unicast.receive] Mensagem recebida: {'tid': '882f6960-3ad8-4a03-a4f4-65a20bffb47d', 'status': 'COMMITED'}
[Cliente 4050] Efetuando SCAN no datastore
[Unicast.request] Enviando para localhost:4001 a mensagem {'type': 'SCAN', 'cid': 4050}
{'gui': (123, 0), 'x': (100, 1), 'y': (30, 0), 'z': (50, 0), 'teste': (1, 1)}
TX1: {'tid': '882f6960-3ad8-4a03-a4f4-65a20bffb47d', 'status': 'COMMITED'}


In [18]:
from utils.transaction import Transaction

t = Transaction(cid=4050)
print(t.replica)

x = t.read("x", think_time=0)
print(x)

scan = t.scan(think_time=0)
print(scan)

t.write("teste", 1)
t.commit()

scan = t.scan(think_time=0)
print(scan)
print(f"TX1: {t.result}")


('localhost', 4002)
[Cliente 4050] Efetuando leitura na chave x
[Cliente 4050] Valor não encontrado localmente, requisitando valor ao banco
[Unicast.request] Enviando para localhost:4002 a mensagem {'type': 'READ', 'key': 'x', 'cid': 4050}
('x', 20, 1)
[Cliente 4050] Efetuando SCAN no datastore
[Unicast.request] Enviando para localhost:4002 a mensagem {'type': 'SCAN', 'cid': 4050}
{'gui': (123, 1), 'x': (20, 1), 'y': (30, 1), 'z': (50, 1), 'teste': (1, 6)}
[Cliente 4050] Efetuando escrita na chave 'teste' com valor '1'
[Cliente 4050] Commitando transação 7a62d956-b72d-4747-9283-3533e42af707
[ABCAST] Enviando para localhost:4000 a mensagem {'type': 'COMMIT_REQ', 'cid': 4050, 'tid': '7a62d956-b72d-4747-9283-3533e42af707', 'rs': [('x', 20, 1)], 'ws': [('teste', 1)]}
[Cliente 4050] Transação já finalizou com resultado: {'tid': 'f1c33f7c-8cc7-4b8d-9ccf-7a9b1f644fd5', 'status': 'COMMITED'}
('7a62d956-b72d-4747-9283-3533e42af707', {'tid': 'f1c33f7c-8cc7-4b8d-9ccf-7a9b1f644fd5', 'status': 'COM

In [28]:
from utils.unicast import request

def scan_replica(replica):
    response = request(*replica, {'type': "SCAN", 'cid': 4610})
    response = {k: tuple(v) for k, v in response.items()} # JSON não suporta tuplas, dessa forma é necessário normalizar
    return response

In [33]:
from utils import load_replicas

for host, read_port, _ in load_replicas():
    print(replica)
    result = scan_replica((host, read_port))
    print(result)

('localhost', 4001, 5001)
[Unicast.request] Enviando para localhost:4001 a mensagem {'type': 'SCAN', 'cid': 4610}
{'gui': (123, 1), 'x': (100, 67), 'y': (30, 1), 'z': (50, 1)}
('localhost', 4001, 5001)
[Unicast.request] Enviando para localhost:4002 a mensagem {'type': 'SCAN', 'cid': 4610}
{'gui': (123, 1), 'x': (100, 67), 'y': (30, 1), 'z': (50, 1)}
('localhost', 4001, 5001)
[Unicast.request] Enviando para localhost:4003 a mensagem {'type': 'SCAN', 'cid': 4610}
{'gui': (123, 1), 'x': (100, 67), 'y': (30, 1), 'z': (50, 1)}


In [5]:
from utils import load_replicas, get_sequencer

In [6]:
load_replicas()

[('localhost', '4001', '5001'),
 ('localhost', '4002', '5002'),
 ('localhost', '4003', '5003')]

In [4]:
get_sequencer()

('localhost', '4000')

In [7]:
list(config)

['DEFAULT', 'sequencer', 'replica_1', 'replica_2', 'replica_3', 'replica_4']

In [8]:
config['DEFAULT']

<Section: DEFAULT>

In [9]:
config.sections()

['sequencer', 'replica_1', 'replica_2', 'replica_3', 'replica_4']

In [16]:
for section in config.sections() if section != 'sequencer':
    print(section)

SyntaxError: invalid syntax (1697189590.py, line 1)

In [15]:
for section in config.sections():
    print(config[section]['port'])

4000
4001
4002
4003
4004


In [ ]:
# Acessa valores
host = config['database']['host']
port = int(config['database']['port'])
debug = config.getboolean('app', 'debug')

print(f"Conectando ao banco em {host}:{port}")
print(f"Modo debug: {debug}")